In [ ]:
# !pip install librosa

In [ ]:
from google.colab import drive
import pandas as pd
import json


# 구글 드라이브 마운트
drive.mount('/content/drive')

In [ ]:
path_audio="/content/drive/MyDrive/프로젝트/2025_과외_프로젝트1/data/TS_in_4"

In [ ]:
path_audio

In [ ]:
path_text="/content/drive/MyDrive/프로젝트/2025_과외_프로젝트1/data/TL_out"

In [ ]:
label=pd.read_csv('/content/drive/MyDrive/프로젝트/2025_과외_프로젝트1/data/emotions.csv')

## text(json)

In [ ]:
import os

all_files_json=os.listdir(path_text)

In [ ]:
len(all_files_json)

In [ ]:
# 파일 이름 넘버
# type A만이 아니고 전체 텍스트
# 첫 start
# 끝 last


import json

all_data=[]

for i in range(len(all_files_json)):

    file_name=os.path.join(path_text,all_files_json[i])

    with open(file_name, 'r', encoding='utf-8') as f:
        data=json.load(f)

        # print(data['info']['ID'])
        # print(data['list'])
        # print(len(data['list']))



        temp={}
        text=[]
        start=[]
        end=[]

        for p in range(len(data['list'])):
            for q in range(len(data['list'][p]['list'])):


                data_lst=data['list'][p]['list'][q]


                if 'audio' in data_lst:

                    for i in range(len(data_lst['audio'])):
                        if data_lst['audio'][i]['type']=='A':
                            start.append(data_lst['audio'][i]['start'])
                            end.append(data_lst['audio'][i]['end'])
                            text.append(data_lst['audio'][i]['text'])


                else:
                    text.append('')

        temp['id']=data['info']['ID']
        temp['start']=start
        temp['end']=end
        temp['text']=text

    all_data.append(temp)


In [ ]:
text_data=pd.DataFrame(all_data)

In [ ]:
text_data['text']=text_data['text'].apply(lambda x: ' '.join(x))

In [ ]:
text_data[text_data['id']=='0002']['text']

## audio

In [ ]:
import os

all_files=os.listdir(path_audio)

audio_data={}
mp3_files=[]
id_lst=[]

for f in all_files:
    if f.endswith('.mp3'):
        mp3_files.append(f)

        id=f.replace('.mp3',"")
        id_lst.append(id)

audio_data['id']=id_lst
audio_data['mp3_files']=mp3_files

audio_data=pd.DataFrame(audio_data)


## label

In [ ]:
label['emotion'] = label['emotion'].replace({
    '슬픈': '슬픔',
    '행복': '기쁨',
    '두려움': '불안',
    '혐오': '상처',
    '중립': '당황'
})

In [ ]:
label['emotion'].value_counts()

In [ ]:
label.head()

In [ ]:
label['id']=label['filename'].apply(lambda x: x.replace('.json',''))

In [ ]:
label=label[['emotion','id']]

In [ ]:
label

### 실제로는 키값으로 join// 그리고 만약 텍스트 데이터 없으면 concate 안하고 음성만 stt로 텍스트 추출

In [ ]:
concat_data=pd.merge(text_data, audio_data, on='id', how='right')

In [ ]:
concat_data=pd.merge(label, concat_data, on='id', how='right')

In [ ]:
concat_data

In [ ]:
concat_data['text'][0]

In [ ]:
concat_data=concat_data.drop(['id'], axis=1)

In [ ]:
concat_data['label']=concat_data['emotion']

In [ ]:
concat_data['mp3_files'][0]

In [ ]:
import librosa
import numpy as np
from tqdm import tqdm
import pandas as pd


n_mfcc = 30 #몇차원 벡터로 뽑아 낼거냐? #보통 딥러닝에서는 30~40사이를 합니다.

In [ ]:
concat_data['mp3_files']

In [ ]:
mfcc_lst=[]
id_lst=[]

for idx in range(len(concat_data)):


    id=concat_data['mp3_files'][idx].replace(".mp3","")

    start_list = concat_data['start'][idx]
    end_list = concat_data['end'][idx]


    voc_path = os.path.join(path_audio, concat_data['mp3_files'][idx])

    data, sr = librosa.load(voc_path, sr = 16000) # 사람목소리는 대부분 16000 안에 포함됨 어느 음성이냐에따라서 주파수라는게 다름.


    #답변 구간들을 추출
    audio_segments = []

    for start_time, end_time in zip(start_list, end_list):

        if '--' in str(start_time) or '--' in str(end_time):
            continue

        try:

            start_min, start_sec = start_time.split(":")
            start_int=float(start_min)*60 + float(start_sec)

            end_min, end_sec = end_time.split(":")
            end_int=float(end_min)*60 + float(end_sec)



            # # "start" ~ "end" 타임스탬프 기준으로 오디오 구간 추출
            start_sr=int(start_int*sr)
            end_sr=int(end_int*sr)

            segment =data[start_sr:end_sr]

            if len(segment) > 0:
                audio_segments.append(segment)

        except Exception as e:
            print(f"⚠️ idx={idx} 오류: {e}")
            continue


    # 유효한 구간이 없으면 건너뛰기
    if len(audio_segments) == 0:
        print(f"❌ idx={idx} ({id}) 건너뜀 - 유효한 구간 없음")
        skipped_count += 1
        continue


    # ✅ 추가: 모든 발화 연결
    data = np.concatenate(audio_segments)


    # 무음 제거(앞뒤 자르기)
    data_trim,temp=librosa.effects.trim(data, top_db=20)
    #print(data_trim)


    # 볼륨 표준화 공식 ( 원본 오디오(trim) / (평균 RMS +epsilon) ) * 목표 RMS
    rms=librosa.feature.rms(y=data_trim)

    mean_rms=np.mean(rms)
    epsilon=1e-10 # rms가 0이 나와 0으로 나누는거 방지하기 위해
    target_rms=0.05  # 0.01 ~ 0.1 사이의 작은 값 사용 너무 크면 왜곡
    data_normalized=(data_trim/(mean_rms+epsilon) * target_rms)


    # mfcc 추출
    mfcc = librosa.feature.mfcc(y=data_normalized, sr=sr, n_mfcc=n_mfcc)

    mfcc_lst.append(mfcc)
    id_lst.append(id)



### 이부분에서 id 기준 join으로 해야 하나? ###
concat_data['mfcc']=mfcc_lst
concat_data['id']=id_lst


In [ ]:
concat_data['mfcc'][1]

In [ ]:
concat_data_4=concat_data

In [ ]:
# id 기준으로 DataFrame 전체 정렬
concat_data_4 = concat_data_4.sort_values('id').reset_index(drop=True)

In [ ]:
concat_data_4.head()

In [ ]:
concat_data_4.info()

In [ ]:

# pickle로 저장
import pickle
concat_data_4.to_pickle('/content/drive/MyDrive/프로젝트/2025_과외_프로젝트1/data/음성데이터전처리_4.pkl')

print("✅ pickle로 저장 완료!")

In [ ]:
# 이제 바로 사용 가능!
print(f"✅ MFCC 타입: {type(concat_data_4['mfcc'].iloc[0])}")
print(f"✅ MFCC shape: {concat_data_4['mfcc'].iloc[0].shape}")